In [1]:
# Install Hugging Face Transformers and spaCy
!pip -q install -U spacy transformers
import sys, subprocess
subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 66.5 MB/s eta 0:00:00


CompletedProcess(args=['/usr/bin/python3', '-m', 'spacy', 'download', 'en_core_web_sm'], returncode=0)

In [2]:
import spacy
from transformers import BertTokenizerFast
# Load spaCy pipeline
nlp = spacy.load("en_core_web_sm")
# Load Hugging Face tokenizer (BERT WordPiece)
hf_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
def show_spacy_tokens(text: str):
    doc = nlp(text)
    print("\nspaCy tokens")
    print("-" * 50)
    for i, tok in enumerate(doc):
        print(f"{i:>2}  {tok.text:<15}  ({tok.idx}, {tok.idx+len(tok)})")
    print("-" * 50)

def show_hf_tokens(text: str, add_special_tokens=True):
    enc = hf_tokenizer(text, return_offsets_mapping=True,return_special_tokens_mask=True, add_special_tokens=add_special_tokens)
    tokens = hf_tokenizer.convert_ids_to_tokens(enc["input_ids"])
    offsets = enc["offset_mapping"]
    special_mask = enc["special_tokens_mask"]

    print("\nHugging Face tokens")
    print("-" * 50)
    for i, (tok, (s, e), sp) in enumerate(zip(tokens, offsets, special_mask)):
        print(f"{i:>2}  {tok:<15}  ({s}, {e})   special={bool(sp)}")
    print("-" * 5)


In [6]:
text = "He went to the bank to deposit money."
show_spacy_tokens(text)
show_hf_tokens(text, add_special_tokens=True)



spaCy tokens
--------------------------------------------------
 0  He               (0, 2)
 1  went             (3, 7)
 2  to               (8, 10)
 3  the              (11, 14)
 4  bank             (15, 19)
 5  to               (20, 22)
 6  deposit          (23, 30)
 7  money            (31, 36)
 8  .                (36, 37)
--------------------------------------------------

Hugging Face tokens
--------------------------------------------------
 0  [CLS]            (0, 0)   special=True
 1  he               (0, 2)   special=False
 2  went             (3, 7)   special=False
 3  to               (8, 10)   special=False
 4  the              (11, 14)   special=False
 5  bank             (15, 19)   special=False
 6  to               (20, 22)   special=False
 7  deposit          (23, 30)   special=False
 8  money            (31, 36)   special=False
 9  .                (36, 37)   special=False
10  [SEP]            (0, 0)   special=True
-----


In [7]:
samples = [
    "I don't know why it's like that.",
    "Email me at hello@example.com or visit https://example.com now!",
    "State-of-the-art models are cool.",
    "Caffè latte 😊 is tasty.",
    "Unbelievable performances!!!"
]

for s in samples:
    show_spacy_tokens(s)
    show_hf_tokens(s, add_special_tokens=True)



spaCy tokens
--------------------------------------------------
 0  I                (0, 1)
 1  do               (2, 4)
 2  n't              (4, 7)
 3  know             (8, 12)
 4  why              (13, 16)
 5  it               (17, 19)
 6  's               (19, 21)
 7  like             (22, 26)
 8  that             (27, 31)
 9  .                (31, 32)
--------------------------------------------------

Hugging Face tokens
--------------------------------------------------
 0  [CLS]            (0, 0)   special=True
 1  i                (0, 1)   special=False
 2  don              (2, 5)   special=False
 3  '                (5, 6)   special=False
 4  t                (6, 7)   special=False
 5  know             (8, 12)   special=False
 6  why              (13, 16)   special=False
 7  it               (17, 19)   special=False
 8  '                (19, 20)   special=False
 9  s                (20, 21)   special=False
10  like             (22, 26)   special=False
11  that             (27,

In [8]:
text = "Unbelievable performances!!!"

doc = nlp(text)
print("spaCy offsets:")
for i, tok in enumerate(doc):
    print(i, tok.text, (tok.idx, tok.idx + len(tok)))

enc = hf_tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
tokens = hf_tokenizer.convert_ids_to_tokens(enc["input_ids"])
print("\nHF offsets:")
for i, (tok, (s, e)) in enumerate(zip(tokens, enc["offset_mapping"])):
    print(i, tok, (s, e))



spaCy offsets:
0 Unbelievable (0, 12)
1 performances (13, 25)
2 ! (25, 26)
3 ! (26, 27)
4 ! (27, 28)

HF offsets:
0 unbelievable (0, 12)
1 performances (13, 25)
2 ! (25, 26)
3 ! (26, 27)
4 ! (27, 28)


In [9]:
text = "Unbelievable performances!!!"
enc = hf_tokenizer(text, add_special_tokens=False)
wp_tokens = hf_tokenizer.convert_ids_to_tokens(enc["input_ids"])

def merge_wordpieces(tokens):
    words, current = [], ""
    for t in tokens:
        if t.startswith("##"):
            current += t[2:]
        else:
            if current:
                words.append(current)
            current = t
    if current:
        words.append(current)
    return words

print("WordPiece tokens:", wp_tokens)
print("Reconstructed words:", merge_wordpieces(wp_tokens))


WordPiece tokens: ['unbelievable', 'performances', '!', '!', '!']
Reconstructed words: ['unbelievable', 'performances', '!', '!', '!']


In [11]:
def count_tokens_spacy(text):
    return len(nlp(text))

def count_tokens_hf(text):
    return len(hf_tokenizer(text, add_special_tokens=False)["input_ids"])

texts = [
    "He went to the bank to deposit money.",
    "Unbelievable performances!!!",
    "Caffè latte 😊 is tasty.",
    "State-of-the-art models are cool."
]

print(f"{'Text':<40} {'spaCy':>6} {'HF Subwords':>12}")
print("-" * 60)
for t in texts:
    print(f"{t:<40} {count_tokens_spacy(t):>6} {count_tokens_hf(t):>12}")



Text                                      spaCy  HF Subwords
------------------------------------------------------------
He went to the bank to deposit money.         9            9
Unbelievable performances!!!                  5            5
Caffè latte 😊 is tasty.                       6            9
State-of-the-art models are cool.            11           11
